# OpenAI Responses API: Advanced Tweet Analysis with File & Web Search Integration

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Connecting to a MongoDB database** to store and retrieve tweets.
- **Extracting tweets** and converting them into a structured format for further analysis.
- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval 
and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API and interact with a MongoDB database, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API
- **`pymongo`**: A Python driver for MongoDB to store and retrieve tweets.

In [1]:
pip install openai pymongo -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Connect to MongoDB

In [4]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
# tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

## Extract Tweets from MongoDB

In [5]:
filter={

    
}
project={
    'tweet.text': 1,
    '_id':0
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

After retrieving tweets from MongoDB, we convert the query result into a list format for easier processing.
The data is then serialized into a JSON-formatted string, ensuring it can be properly stored and shared across different services.
Using `io.BytesIO`, we create an in-memory JSON file, eliminating the need for disk writes.
This approach is particularly useful for applications that require temporary file storage, such as uploading datasets
to OpenAI's file search API or cloud storage for further analysis.

In [6]:
result_list = list(result)

# Convert result list to JSON string
json_data = json.dumps(result_list, default=str, indent=4)

# Create an in-memory JSON file
json_bytes = io.BytesIO(json_data.encode("utf-8"))
json_bytes.name = "tweet.json" 

In [7]:
print('Number of tweets: ',len(result_list))

Number of tweets:  298


## Initialize OpenAI Client

In [8]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [9]:
vector_store = client.vector_stores.create(
    name="tweet_base"
)
vector_store_id = vector_store.id
# print(vector_store_id)

### Upload Tweets File

In [10]:
file = client.files.create(
            file=json_bytes,
            purpose="assistants",)

file_id = file.id
# print(file_id)

### Attach File to Vector Store

In [18]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

# print(attach_status.id)

### Query the Vector Store

In [19]:
query = "jmu sport"

In [20]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

{
        "tweet": {
            "text": "Sai shiye walt\u01ddyiShim adamganabe nyiro kara\n\n\u2390
 Relevant score: 0.5347618399387305
\n.\n.\n.\n.\n.\njMu\u062d"
        }
    },
    {
        "tweet": {
            "text": "Ya K\u01d
 Relevant score: 0.45519906961360856
{
        "tweet": {
            "text": "RT @LongwoodBase: Here\u2019s our starting lineup against 
 Relevant score: 0.44616348033953956
\u0628\u0624\u062f\u0649\u2390\n\u22b5A4TU\u22b4\nJMu"
        }
    },
    {
        "tweet": {
   
 Relevant score: 0.44232641115993143
{
        "tweet": {
            "text": "Sai shiye walt\u01ddyi\n\n\u2390\u0628\u0644\u0624\u0645\u
 Relevant score: 0.4393625698813693


## OpenAI Response API

### Simple Response

In [21]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [22]:
display(Markdown(simple_response.output_text))

James Madison University (JMU) offers a diverse and competitive athletics program. Known as the Dukes, JMU's sports teams participate in the Sun Belt Conference at the NCAA Division I level. Here's a brief overview of JMU sports:

1. **Football**: JMU has a strong football program with a proud history of success, including a national championship in the Football Championship Subdivision (FCS).

2. **Basketball**: Both men's and women's basketball teams are competitive and attract significant attention.

3. **Baseball and Softball**: The Dukes' baseball and softball programs are known for their talent and skill, competing at high levels within the conference.

4. **Soccer**: JMU offers both men's and women's soccer teams that compete strongly in the Sun Belt Conference.

5. **Other Sports**: Additionally, JMU hosts teams in sports like field hockey, swimming and diving, track and field, lacrosse, and more.

The university's facilities, such as Bridgeforth Stadium for football and the JMU Convocation Center for basketball, contribute to a vibrant sports atmosphere. JMU is also recognized for its passionate fan base, making game days a thrilling experience.

### File Search Response

In [23]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [24]:
display(Markdown(file_search_response.output_text))


The search results contain various tweets mentioning JMU, which likely refers to James Madison University. The tweets discuss topics such as JMU sports events, including football games and spring practices, as well as other activities related to the university.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [28]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= 'the latest news on jmu football',
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [29]:
display(Markdown(web_search_response.output_text))

As of April 21, 2025, James Madison University's (JMU) football program has experienced several notable developments:

**Coaching Changes:**
- In December 2024, Bob Chesney was appointed as the new head coach of the Dukes. Chesney previously led Holy Cross to significant success, including multiple Patriot League titles. ([sports.yahoo.com](https://sports.yahoo.com/holy-cross-football-coach-bob-151736024.html?utm_source=openai))

**2024 Season Highlights:**
- Under Chesney's leadership, JMU concluded the 2024 season with a 9–4 record, including a 4–4 mark in Sun Belt Conference play. ([en.wikipedia.org](https://en.wikipedia.org/wiki/2024_James_Madison_Dukes_football_team?utm_source=openai))
- The Dukes secured a victory in the Boca Raton Bowl, defeating Western Kentucky 27–17. ([en.wikipedia.org](https://en.wikipedia.org/wiki/2024_James_Madison_Dukes_football_team?utm_source=openai))

**Notable Games:**
- On November 16, 2024, JMU edged out Old Dominion with a 35-32 win. Quarterback Alonza Barnett III was instrumental, throwing a 68-yard touchdown pass and rushing for two additional touchdowns. ([wtop.com](https://wtop.com/ncaa-football/2024/11/alonza-barnett-iii-accounts-for-3-tds-james-madison-beats-old-dominion-35-32/?utm_source=openai))

**Upcoming Games:**
- Looking ahead, JMU has scheduled a game at Louisville on September 6, 2025. Additionally, their matchup at Virginia Tech has been rescheduled to September 26, 2026. ([jmusports.com](https://jmusports.com/news/2023/10/23/football-jmu-to-play-at-louisville-in-2025-virginia-tech-moved-to-2026.aspx?utm_source=openai))

These developments reflect JMU's ongoing commitment to strengthening its football program and competing at a high level in collegiate football. 

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [30]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

As of April 21, 2025, James Madison University's (JMU) football program has experienced several nota

### Continue Query with Web Search

In [31]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [32]:
display(Markdown(continue_search_response.output_text))

Certainly, here are some additional updates on James Madison University's (JMU) football program:

**Coaching Transition:**
- On November 30, 2023, Head Coach Curt Cignetti resigned to accept a position at Indiana University. ([jmusports.com](https://jmusports.com/news/2023/11/30/curt-cignetti-resigns-as-head-coach-of-jmu-football?utm_source=openai))
- Bob Chesney was appointed as the new head coach on December 7, 2023. Chesney previously led Holy Cross to five consecutive Patriot League Championships and brings a career record of 111-46 to JMU. ([jmu.edu](https://www.jmu.edu/news/2023/12/07-chesney-hired.shtml?utm_source=openai))

**Player Transfers:**
- Linebacker Jailin Walker entered the NCAA transfer portal after the 2023 season and transferred to Indiana University, where he became a starting linebacker for the Hoosiers in the 2024 season. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Jailin_Walker?utm_source=openai))
- Several high school recruits decommitted from JMU following Cignetti's departure, including four-star wide receiver Jah Jah Boyd, who later committed to Indiana. ([breezejmu.org](https://www.breezejmu.org/sports/the-breeze-tracks-the-ncaa-transfer-portal-s-impact-on-jmu-football/article_279c84bc-90bb-11ee-aaa1-cfada26dabd3.html?utm_source=openai))

**Bowl Game Eligibility:**
- In November 2023, discussions arose regarding JMU's eligibility for bowl games during their transition to the Football Bowl Subdivision (FBS). Despite a strong season, the NCAA's reclassification rules initially limited their bowl participation. ([washingtonpost.com](https://www.washingtonpost.com/sports/2023/11/08/jmu-bowl-game-ncaa/?utm_source=openai))

**Stadium Attendance Records:**
- Bridgeforth Stadium set a new attendance record on October 28, 2023, with 26,239 fans witnessing JMU's 30-27 victory over Old Dominion. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Bridgeforth_Stadium?utm_source=openai))

These developments highlight a period of significant change and growth for JMU football as they continue to establish themselves in the FBS landscape. 

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [33]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [34]:
display(Markdown(combined_search_response.output_text))

James Madison University's (JMU) athletic programs have recently achieved significant milestones across multiple sports:

**Football:**
- The Dukes have announced their 2025 football schedule, featuring 12 games evenly split between home and away matchups. ([bleacherreport.com](https://bleacherreport.com/james-madison-football?utm_source=openai))
- In a notable game, JMU defeated the University of North Carolina with a commanding score of 70-20. ([bleacherreport.com](https://bleacherreport.com/james-madison-football/?utm_source=openai))

**Men's Basketball:**
- The team secured the Sun Belt Conference title and earned a spot in the NCAA tournament by defeating their opponent 91-71 in the championship game. ([richmond.com](https://richmond.com/sports/college/schools/james-madison/?utm_source=openai))

**Athletic Department Updates:**
- JMU is preparing for potential changes in scholarship allocations and roster sizes across various sports, including football and basketball, in response to proposed NCAA settlements. ([dnronline.com](https://www.dnronline.com/sports/dukes-preparing-for-potential-scholarship-roster-changes/article_e35ad5be-1930-5b3e-b203-0db9a70bca18.html?utm_source=openai))

These developments highlight JMU's ongoing commitment to excellence in collegiate athletics. 